# Results

In [1]:
import sys
import time
import pandas as pd

sys.path.append('../')
import lib.functions as f
import trace_analysis
import plots_analysis
import trace_statistics
import trace_classification
import warnings
warnings.filterwarnings('ignore')


## Data Preparation

In [2]:
directory="../cooja3-16nodes/"
df = pd.read_csv(directory + "/traces/traces.csv", sep=',', encoding='utf-8')


win_25_stats=f.create_stats(directory,df,pings=100,window=25)
win_50_stats=f.create_stats(directory,df,pings=100,window=50)
win_100_stats=f.create_stats(directory,df,pings=100,window=100)
win_200_stats=f.create_stats(directory,df,pings=200,window=200)
trace_stats = {
    25: win_25_stats.drop(columns=["label_2"]), 
    50: win_50_stats.drop(columns=["label_2"]), 
    100: win_100_stats.drop(columns=["label_2"]),
    200: win_200_stats.drop(columns=["label_2"])
}

net_win_25_stats = f.create_network_stats(win_25_stats).drop(columns=["label_2"])
net_win_50_stats = f.create_network_stats(win_50_stats).drop(columns=["label_2"])
net_win_100_stats = f.create_network_stats(win_100_stats).drop(columns=["label_2"])
net_win_200_stats = f.create_network_stats(win_100_stats).drop(columns=["label_2"])

# Create a dictionary containing all the statistics for each trace size
network_stats = {200: net_win_200_stats, 25: net_win_25_stats, 50: net_win_50_stats, 100: net_win_100_stats}


features_to_drop = [
    'node_id', 'experiment', 'label',"window",
    "mean",
    #'loss', 'count', 'outliers'
    "hop",
    #"min","max"
]
net_features_to_drop = ['experiment', 'label']
results_2_classes_node,results_2_classes_network=f.results_2_classes(trace_stats,network_stats,features_to_drop,net_features_to_drop)

Processing...
Processing...
Processing...
Processing...
Computing trace 25
Computing trace 50
Computing trace 100
Computing trace 200
Computing trace 25
Computing trace 50
Computing trace 100
Computing trace 200
Computing trace 200
Computing trace 25
Computing trace 50
Computing trace 100
Computing trace 200
Computing trace 25
Computing trace 50
Computing trace 100
Computing trace 25
Computing trace 50
Computing trace 100
Computing trace 200
Computing trace 25
Computing trace 50
Computing trace 100
Computing trace 200
Computing trace 200
Computing trace 25
Computing trace 50
Computing trace 100
Computing trace 200
Computing trace 25
Computing trace 50
Computing trace 100
Computing trace 25
Computing trace 50
Computing trace 100
Computing trace 200
Computing trace 200
Computing trace 25
Computing trace 50
Computing trace 100


In [2]:
directory="../cooja3-16nodes/"
df = pd.read_csv(directory + "/traces/traces.csv", sep=',', encoding='utf-8')


win_25_stats=f.create_stats(directory,df,pings=100,window=25).drop(columns="label").rename(columns={"label_2":"label"})
win_50_stats=f.create_stats(directory,df,pings=100,window=50).drop(columns="label").rename(columns={"label_2":"label"})
win_100_stats=f.create_stats(directory,df,pings=100,window=100).drop(columns="label").rename(columns={"label_2":"label"})
win_200_stats=f.create_stats(directory,df,pings=200,window=200).drop(columns="label").rename(columns={"label_2":"label"})
trace_stats_3_classes = {
    25: win_25_stats, 
    50: win_50_stats, 
    100: win_100_stats,
    200: win_200_stats
}
win_200_stats

Processing...
Processing...
Processing...
Processing...


,experiment,node_id,label,loss,count,std,mean,var,hop,min,max,outliers,window
0,grid_normal_2019-02-26_11:48_,aaaa::212:740e:e:e0e,Normal,63.0,74,666.053366,1055.189189,4.436271e+05,5,407.0,3571.0,3,200
1,grid_normal_2019-02-26_11:48_,aaaa::212:7408:8:808,Normal,60.5,79,528.846069,791.164557,2.796782e+05,2,168.0,2833.0,3,200
2,grid_normal_2019-02-26_11:48_,aaaa::212:740d:d:d0d,Normal,57.5,85,810.198302,1126.858824,6.564213e+05,4,363.0,5397.0,4,200
3,grid_normal_2019-02-26_11:48_,aaaa::212:7403:3:303,Normal,45.0,110,387.171808,394.785455,1.499020e+05,1,61.0,1877.0,6,200
4,grid_normal_2019-02-26_11:48_,aaaa::212:740a:a:a0a,Normal,60.5,79,755.269843,1116.506329,5.704325e+05,4,335.0,4001.0,5,200
5,grid_normal_2019-02-26_11:48_,aaaa::212:740f:f:f0f,Normal,56.5,87,972.063208,1409.655172,9.449069e+05,4,363.0,5619.0,4,200
6,grid_normal_2019-02-26_11:48_,aaaa::212:7410:10:1010,Normal,71.0,58,820.395899,1299.275862,6.730494e+05,4,514.0,4358.0,4,200
7,grid_normal_2019-02-26_11:48_,aaaa::212:740c:c:c0c,Normal,53.0,94,822.630679,1109.329787,6.767212e+05,3,251.0,4432.0,4,200
8,grid_normal_2019-02-26_11:48_,aaaa::212:7404:4:404,Normal,62.5,75,331.593488,379.024000,1.099542e+05,1,65.6,1551.0,3,200
9,grid_normal_2019-02-26_11:48_,aaaa::212:7406:6:606,Normal,51.5,97,503.609470,902.793814,2.536225e+05,3,236.0,2790.0,4,200


In [3]:
net_win_25_stats = f.create_network_stats(win_25_stats).drop(columns=["label_2"])
net_win_50_stats = f.create_network_stats(win_50_stats).drop(columns=["label_2"])
net_win_100_stats = f.create_network_stats(win_100_stats).drop(columns=["label_2"])
net_win_200_stats = f.create_network_stats(win_100_stats).drop(columns=["label_2"])

# Create a dictionary containing all the statistics for each trace size
network_stats_3_classes = {200: net_win_200_stats, 25: net_win_25_stats, 50: net_win_50_stats, 100: net_win_100_stats}


features_to_drop = [
    'node_id', 'experiment', 'label',"window",
    "mean",
    #'loss', 'count', 'outliers'
    "hop",
    #"min","max"
]
net_features_to_drop = ['experiment', 'label']
def results_3_classes(trace_stats,network_stats,features_to_drop,net_features_to_drop):
    results,net_results=f.results_2_classes(trace_stats,network_stats,features_to_drop,net_features_to_drop)
    print(results)
    results=results.reset_index(drop=True)
    net_results=results.reset_index(drop=True)
    net_results=net_results[net_results["Model"]!="Kmeans"]
    results=results[results["Model"]!="Kmeans"]
    return results,net_results

#print(network_stats[100].drop(columns=net_features_to_drop))
results_3_classes_node,results_3_classes_network=results_3_classes(trace_stats_3_classes,network_stats_3_classes,features_to_drop,net_features_to_drop)

Computing trace 25
Computing trace 50
Computing trace 100
Computing trace 200
Computing trace 25
Computing trace 50
Computing trace 100
Computing trace 200
Computing trace 200
Computing trace 25
Computing trace 50
Computing trace 100
Computing trace 200
Computing trace 25
Computing trace 50
Computing trace 100
Computing trace 25
Computing trace 50
Computing trace 100
Computing trace 200
Computing trace 25
Computing trace 50
Computing trace 100
Computing trace 200
Computing trace 200
Computing trace 25
Computing trace 50
Computing trace 100
Computing trace 200
Computing trace 25
Computing trace 50
Computing trace 100
Computing trace 25
Computing trace 50
Computing trace 100
Computing trace 200
Computing trace 200
Computing trace 25
Computing trace 50
Computing trace 100
    Accuracy  F1-score          Model  Precision    Recall  \
0   0.780000  0.756943  Random Forest   0.764518  0.751849   
1   0.736000  0.709118  Random Forest   0.704871  0.716173   
2   0.603175  0.521713  Random For

## Data Preparation Corrected

In [7]:
directory="../cooja3-16nodes/"
df = pd.read_csv(directory+"stats_corrected2.csv")
df=f.correct_stats(df,directory)
df3=df.drop(columns="label").rename(columns={"label_2":"label"})
df2=df.rename(columns={"label":"label"}).drop(columns="label_2")


win_25_stats_c=df2[df2["window"]==25].drop(columns=["mean","window"])
win_50_stats_c=df2[df2["window"]==50].drop(columns=["mean","window"])
win_100_stats_c=df2[df2["window"]==100].drop(columns=["mean","window"])
trace_stats_corrected_2_classes = {25: win_25_stats_c, 50: win_50_stats_c, 100: win_100_stats_c}

win_25_stats_c=df3[df3["window"]==25].drop(columns=["mean","window"])
win_50_stats_c=df2[df3["window"]==50].drop(columns=["mean","window"])
win_100_stats_c=df3[df3["window"]==100].drop(columns=["mean","window"])
trace_stats_corrected_3_classes= {25: win_25_stats_c, 50: win_50_stats_c, 100: win_100_stats_c}

In [9]:

features_to_drop = ['node_id', 'experiment', 
                  
                  "std",
                  #"var",
                    'label', 
                    'hop', 
                    #'loss',
                    'count', 'outliers'
                   ]
net_features_to_drop = ['experiment', 'label']

results_2_classes_node_corrected,_=f.results_2_classes(trace_stats_corrected_2_classes,network_stats,features_to_drop,net_features_to_drop)
results_3_classes_node_corrected,_=results_3_classes(trace_stats_corrected_3_classes,network_stats_3_classes,features_to_drop,net_features_to_drop)

Computing trace 25
Computing trace 50
Computing trace 100
Computing trace 25
Computing trace 50
Computing trace 100
Computing trace 200
Computing trace 25
Computing trace 50
Computing trace 100
Computing trace 200
Computing trace 25
Computing trace 50
Computing trace 100
Computing trace 25
Computing trace 50
Computing trace 100
Computing trace 25
Computing trace 50
Computing trace 100
Computing trace 200
Computing trace 25
Computing trace 50
Computing trace 100
Computing trace 200
Computing trace 25
Computing trace 50
Computing trace 100
Computing trace 25
Computing trace 50
Computing trace 100
Computing trace 200
Computing trace 25
Computing trace 50
Computing trace 100
Computing trace 25
Computing trace 50
Computing trace 100
Computing trace 25
Computing trace 50
Computing trace 100
Computing trace 25
Computing trace 50
Computing trace 100
Computing trace 200
Computing trace 25
Computing trace 50
Computing trace 100
Computing trace 200
Computing trace 25
Computing trace 50
Computing 

Problem 1.1 (easy): Given a set of traces collected from the network, we want to characterize if the whole network includes at least 1 malicious node.

Results:

- Using K-means based on per-node packet loss, we can achieve high accuracy.

- Performance of K-means for 50, 200 sized traces.
 



Problem 1.2: Given a set of traces collected from the network, we want to characterize if the whole network includes at least 1 malicious node and also identify the type of attack.



- Using K-means based on per-node packet loss, the accuracy is not high.
 



Problem 2.1: Given a set of traces collected from the network, we want to characterize each node in the network if it is a malicious node.

Problem 2.2: Given a set of traces collected from the network, we want to characterize each node in the network if it is affected by a malicious node.

Problem 2.3: Given a set of traces collected from the network, we want to characterize each node in the network is a malicious node and also identify the type of attack.

Problem 2.4: Given a set of traces collected from the network, we want to characterize each node in the network if it is affected by a malicious node and also identify the type of attack.

Import dimension is the number of packets that consistute the trace. We wish that the traces are minimized.



- Grid 3x3, Grid 4x4 -- Table with statistics: packet loss, average RTT, ...

-  Grid 3x3 -- GH 30\% and GH 70\%

- KMeans Table with accuracy: 
 


## Problem 1.1
Given a set of traces collected from the network, we want to characterize if the whole network includes at least 1 malicious node.

## Solution 1.1
Possible with the accuracy

In [11]:
results_2_classes_network

,Accuracy,F1-score,Model,Precision,Recall,Testing Time (sec),Time (ms),Training Time (sec),Window Size
0,0.250000,0.200000,Random Forest,0.125000,0.500000,0.005556,NaN,0.045824,200
1,0.312500,0.228571,Random Forest,0.224242,0.324074,0.005344,NaN,0.049030,25
2,0.125000,0.095238,Random Forest,0.333333,0.055556,0.005719,NaN,0.048290,50
3,0.250000,0.200000,Random Forest,0.125000,0.500000,0.005289,NaN,0.054870,100
4,0.250000,0.200000,KNN,0.125000,0.500000,0.002265,NaN,0.001032,200
5,0.562500,0.632323,KNN,0.650794,0.648148,0.001563,NaN,0.000684,25
6,0.250000,0.194444,KNN,0.233333,0.388889,0.001077,NaN,0.000617,50
7,0.250000,0.200000,KNN,0.125000,0.500000,0.000835,NaN,0.000582,100
8,0.538462,0.350000,Kmeans,0.500000,0.269231,NaN,0.011161,NaN,200
9,0.557692,0.358025,Kmeans,0.500000,0.278846,NaN,0.012778,NaN,25


## Problem 1.2
Given a set of traces collected from the network, we want to characterize if the whole network includes at least 1 malicious node and also identify the type of attack.

## Solution 1.2
Possible but with low accuracy

In [5]:
results_3_classes_network


,Accuracy,F1-score,Model,Precision,Recall,Testing Time (sec),Time (ms),Training Time (sec),Window Size
0,0.780000,0.756943,Random Forest,0.764518,0.751849,0.007811,NaN,0.089635,25
1,0.736000,0.709118,Random Forest,0.704871,0.716173,0.007550,NaN,0.064014,50
2,0.603175,0.521713,Random Forest,0.601887,0.555556,0.009779,NaN,0.065547,100
3,0.730159,0.716141,Random Forest,0.729490,0.712963,0.006030,NaN,0.058771,200
4,0.696000,0.670139,KNN,0.671017,0.669362,0.009564,NaN,0.002663,25
5,0.648000,0.624932,KNN,0.624738,0.638211,0.007214,NaN,0.001115,50
6,0.603175,0.521713,KNN,0.601887,0.555556,0.002249,NaN,0.000844,100
7,0.682540,0.672557,KNN,0.675263,0.671296,0.002675,NaN,0.000948,200


## Problem 2.1
Given a set of traces collected from the network, we want to characterize each node in the network if it is a malicious node. 

## Solution 2.1
Not possible to identify the specific node, we can just identify if a node is affected but not a malicious node, we can also identify the hop of the malicious node that will be the minimum of the hop of the affected nodes minus 1

## Problem 2.2
Given a set of traces collected from the network, we want to characterize each node in the network if it is affected by a malicious node.


## Solution 2.2
We can understand if the node is affected using a trained model of the network with the nodes affected and we can identify with the following accuracy
- Network trained algorithm
- Node trained algorithm

In [13]:
results_2_classes_node

,Accuracy,F1-score,Model,Precision,Recall,Testing Time (sec),Time (ms),Training Time (sec),Window Size
0,0.804000,0.783458,Random Forest,0.791739,0.777766,0.007679,NaN,0.086133,25
1,0.784000,0.759358,Random Forest,0.755471,0.764373,0.006380,NaN,0.065441,50
2,0.603175,0.535809,Random Forest,0.595588,0.560185,0.007574,NaN,0.061065,100
3,0.714286,0.691848,Random Forest,0.720694,0.689815,0.005738,NaN,0.055402,200
4,0.696000,0.670139,KNN,0.671017,0.669362,0.009100,NaN,0.002523,25
5,0.648000,0.624932,KNN,0.624738,0.638211,0.005468,NaN,0.002391,50
6,0.603175,0.521713,KNN,0.601887,0.555556,0.005625,NaN,0.001295,100
7,0.682540,0.672557,KNN,0.675263,0.671296,0.002934,NaN,0.000857,200
8,0.967548,0.491753,Kmeans,0.500000,0.483774,NaN,0.015823,NaN,25
9,0.949519,0.487053,Kmeans,0.500000,0.474760,NaN,0.014960,NaN,50


In [15]:
results_2_classes_node_corrected

,Accuracy,F1-score,Model,Precision,Recall,Testing Time (sec),Time (ms),Training Time (sec),Window Size
0,0.864000,0.724169,Random Forest,0.741796,0.710414,0.009153,NaN,0.120760,25
1,0.896000,0.746450,Random Forest,0.727064,0.771978,0.007169,NaN,0.077215,50
2,1.000000,1.000000,Random Forest,1.000000,1.000000,0.008259,NaN,0.057045,100
3,0.876000,0.635996,KNN,0.935950,0.602564,0.006005,NaN,0.001019,25
4,0.904000,0.599359,KNN,0.788251,0.572459,0.003513,NaN,0.001009,50
5,0.809524,0.447368,KNN,0.404762,0.500000,0.003937,NaN,0.000682,100
6,0.967548,0.491753,Kmeans,0.500000,0.483774,NaN,0.021413,NaN,25
7,0.949519,0.487053,Kmeans,0.500000,0.474760,NaN,0.022436,NaN,50
8,0.908654,0.476071,Kmeans,0.500000,0.454327,NaN,0.018322,NaN,100


## Problem 2.3
Given a set of traces collected from the network, we want to characterize each node in the network is a malicious node and also identify the type of attack.

## Solution 2.3
It is possible just with the per node trained algorithm, the per network one gives low accuracy

In [4]:
results_3_classes_node

,Accuracy,F1-score,Model,Precision,Recall,Testing Time (sec),Time (ms),Training Time (sec),Window Size
0,0.780000,0.756943,Random Forest,0.764518,0.751849,0.007811,NaN,0.089635,25
1,0.736000,0.709118,Random Forest,0.704871,0.716173,0.007550,NaN,0.064014,50
2,0.603175,0.521713,Random Forest,0.601887,0.555556,0.009779,NaN,0.065547,100
3,0.730159,0.716141,Random Forest,0.729490,0.712963,0.006030,NaN,0.058771,200
4,0.696000,0.670139,KNN,0.671017,0.669362,0.009564,NaN,0.002663,25
5,0.648000,0.624932,KNN,0.624738,0.638211,0.007214,NaN,0.001115,50
6,0.603175,0.521713,KNN,0.601887,0.555556,0.002249,NaN,0.000844,100
7,0.682540,0.672557,KNN,0.675263,0.671296,0.002675,NaN,0.000948,200


In [32]:
results_3_classes_node_corrected

,Accuracy,F1-score,Model,Precision,Recall,Testing Time (sec),Time (ms),Training Time (sec),Window Size
0,0.796000,0.531295,Random Forest,0.526161,0.543184,0.007361,NaN,0.091687,25
1,0.904000,0.758687,Random Forest,0.743939,0.776442,0.008182,NaN,0.083895,50
2,0.984127,0.954751,Random Forest,0.962963,0.952381,0.005679,NaN,0.053309,100
3,0.836000,0.474526,KNN,0.610193,0.449275,0.005707,NaN,0.001074,25
4,0.904000,0.599359,KNN,0.788251,0.572459,0.003318,NaN,0.000900,50
5,0.761905,0.288288,KNN,0.253968,0.333333,0.002303,NaN,0.001135,100
